# 1. Introducing Optim.jl
This notebook introduces the very basic features of Optim.jl, a package for non-linear optimization in Julia. We focus on syntax in this tutorial. More advanced topics are discussed in other notebooks. First of all, we need to install and load Optim.

In [1]:
Pkg.add("Optim")
using Optim

Installing Optim will also install a few other packages, the so-called dependencies. Most of these are not of special interest to you as the end-user, but one is not like the others: LineSearches.jl. We will see later how to use this, and in another notebook we will look closer at the different methods and options related this package.

In a nutshell, Optim is all about optimizing a function $f:\mathbb{R}^N\to\mathbb{R}$ (or by proper transformation also $f:\mathbb{C}^N\to\mathbb{R}$). By optimizing we mean finding an input that makes the function value as small as possible. This does not always come without complications, as we will see below. First, let us consider a specific example: the Himmelblau test function. It is named after David Mautner Himmelblau (1924-2011) who introduced it in his *Applied Nonlinear Programming (1972)* textbook. Mathematically, define
$$
f(x)=(x_1^{2}+x_2-11)^{2}+(x_1+x_2^{2}-7)^{2}
$$
In code, we get the following, that also includes the gradient and Hessian as they will come in handy later.

In [63]:
function himmelblau(x::Vector)
    return (x[1]^2 + x[2] - 11)^2 + (x[1] + x[2]^2 - 7)^2
end

function himmelblau_gradient!(storage::Vector, x::Vector)
    storage[1] = 4.0 * x[1]^3 + 4.0 * x[1] * x[2] -
                  44.0 * x[1] + 2.0 * x[1] + 2.0 * x[2]^2 - 14.0
    storage[2] = 2.0 * x[1]^2 + 2.0 * x[2] - 22.0 +
                  4.0 * x[1] * x[2] + 4.0 * x[2]^3 - 28.0 * x[2]
end

function himmelblau_hessian!(storage::Matrix, x::Vector)
    storage[1, 1] = 12.0 * x[1]^2 + 4.0 * x[2] - 42.0
    storage[1, 2] = 4.0 * x[1] + 4.0 * x[2]
    storage[2, 1] = 4.0 * x[1] + 4.0 * x[2]
    storage[2, 2] = 12.0 * x[2]^2 + 4.0 * x[1] - 26.0
end

himmelblau_hessian! (generic function with 1 method)

To get an idea about the problem we're facing, let us make a plot that shows what choices of $x$ gives high and low values of the objective function.

In [38]:
using Plots
x = -5.0:0.2:5.0
y = -5.0:0.2:5.0
X = repmat(x',length(y),1)
Y = repmat(y,1,length(x))
contour(x,y,(x,y)->himmelblau([x,y]),fill=true, color=:deep)
contour!(x,y,(x,y)->himmelblau([x,y]))
scatter!([3.0, -2.805118, -3.779310, 3.584428], [2.0, 3.131312, -3.283186, -1.848126], legend=false, color=:black)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 600 
 
 
 700 
 
 
 800 
 
 
 900 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 550.315,375.813 550.315,365.511 556.315,365.511 550.315,365.511 550.315,326.927 556.315,326.927 550.315,326.927 550.315,288.343 556.315,288.343 550.315,288.343 
 550.315,249.759 556.315,249.759 550.315,249.759 550.315,211.175 556.315,211.175 550.315,211.175 550.315,172.591 556.315,172.591 550.315,172.591 550.315,134.007 
 556.315,134.007 550.315,134.007 550.315,95.4228 556.315,95.4228 550.315,95.4228 550.315,56.8387 556.315,56.8387 550.315,56.8387 550.315,18.2546 556.315,18.2546 
 550.315,18.2546 550.315,11.811 
 "/>
 
<polyline clip-path="url(#clip02)" style="stroke:#470b69; stroke-width:1; stroke-opacity:1; fill:none" points="
 43.1569,352.415 52.4222,358.011 54.0177,358.644 61.6874,360.981 70.9526,362.542 80.2178,363.062 89.483,362.745 98.7482,361.761 108.013,360.257 115.857,358.644 
 117.279,358.299 126.544,355.669 135.809,352.824 139.028,351.776 145.074,349.446 154.339,345.818 156.653,344.908 163.605,341.647 171.535,338.04 172.87,337.306 
 182.135,332.417 184.646,331.172 191.4,327.089 196.421,324.304 200.665,321.389 207.079,317.436 209.931,315.22 216.774,310.568 219.196,308.444 225.593,303.7 
 228.461,300.88 233.553,296.832 237.726,292.276 240.529,289.964 246.265,283.096 246.991,281.64 251.116,276.228 253.692,269.36 253.829,262.492 251.73,255.624 
 247.736,248.756 246.991,247.703 244.204,241.888 240.871,235.02 238.139,228.152 237.726,225.946 237.07,221.284 237.691,214.416 237.726,214.323 241.657,207.548 
 246.991,202.808 252.303,200.68 256.257,199.602 265.522,200.049 267.222,200.68 274.787,204.458 278.097,207.548 283.137,214.416 284.052,216.532 285.444,221.284 
 286.261,228.152 286.158,235.02 285.463,241.888 284.598,248.756 284.123,255.624 284.816,262.492 287.794,269.36 293.317,274.918 294.229,276.228 302.53,283.096 
 302.583,283.124 311.848,289.586 312.296,289.964 321.113,295.211 323.468,296.832 330.378,300.358 336.239,303.7 339.643,305.191 348.909,309.612 350.795,310.568 
 358.174,313.528 367.439,317.362 367.616,317.436 376.704,320.523 385.97,323.608 388.154,324.304 395.235,326.162 404.5,328.394 413.765,330.337 418.716,331.172 
 423.03,331.781 432.296,332.685 441.561,333.09 450.826,332.899 460.091,332.005 464.602,331.172 469.356,330.122 478.622,326.949 484.112,324.304 487.887,322.089 
 493.854,317.436 497.152,314.23 500.099,310.568 504.402,303.7 506.417,299.41 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#63146e; stroke-width:1; stroke-opacity:1; fill:none" points="
 506.417,320.713 502.819,324.304 497.152,328.922 493.628,331.172 487.887,334.223 478.622,337.938 478.266,338.04 469.356,340.196 460.091,341.649 450.826,342.415 
 441.561,342.592 432.296,342.267 423.03,341.526 413.765,340.445 404.5,339.098 398.149,338.04 395.235,337.465 385.97,335.483 376.704,333.408 367.439,331.306 
 366.835,331.172 358.174,328.86 348.909,326.486 339.779,324.304 339.643,324.264 330.378,321.839 321.113,319.729 311.848,317.994 307.839,317.436 302.583,316.512 
 293.317,315.488 284.052,315.123 274.787,315.482 265.522,316.634 261.766,317.436 256.257,318.395 246.991,320.708 237.726,323.782 236.446,324.304 228.461,326.976 
 219.196,330.701 218.171,331.172 209.931,334.314 201.318,338.04 200.665,338.277 191.4,341.934 184.464,344.908 182.135,345.753 172.87,349.286 166.623,351.776 
 163.605,352.8 154.339,355.994 146.71,358.644 145.074,359.131 135.809,361.804 126.544,364.371 122.054,365.511 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#821f6b; stroke-width:1; stroke-opacity:1; fill:none" points="
 149.761,365.511 154.339,364.107 163.605,361.26 172.333,358.644 172.87,358.457 182.135,355.316 191.4,352.361 19

The in the figure above, we've colored in local minimizers as black dots. The minimum of the Himmelblau function is 0, as $f(x_i)=0$ for all $x_i$ in the set of black dots in the figure (the minimizers). Let us try finding them all, by starting sufficiently close to each one.

In [43]:
# find [3.0, 2.0]
optimize(himmelblau, [2.0,1.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [2.0,1.0]
 * Minimizer: [3.00001352623409,1.999978131760836]
 * Minimum: 8.983274e-09
 * Iterations: 36
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 73

In [50]:
#find [-2.805118, 3.131312]
optimize(himmelblau, [-2.0,1.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-2.0,1.0]
 * Minimizer: [-2.805113428208802,3.131311407053018]
 * Minimum: 7.477355e-10
 * Iterations: 40
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 78

In [49]:
#find [-3.779310, -3.283186]
optimize(himmelblau, [-2.0,-1.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-2.0,-1.0]
 * Minimizer: [-3.779308105087092,-3.283200226675727]
 * Minimum: 1.007246e-08
 * Iterations: 35
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 74

In [51]:
#find [3.584428, -1.848126]
optimize(himmelblau, [2.0,-1.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844298648286093,-1.8481263812232966]
 * Minimum: 1.236197e-10
 * Iterations: 34
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 71

Below, we use an interactive figure to see how these non-linearities come into play. Try keeping $x_2$ fixed at 0.0, and vary $x_1$ to find all the minimizers. Then try to fix $x_1$ at -4.0. Can you trace out all the minimizers by varying $x_2$?

In [84]:
using Interact
@manipulate for xx = slider(-5.0:.5:5.0, value = -2.0, label="x₁"), yy = slider(-5.0:0.5:5.0, value = 0.0, label="x₂")
    _x = [xx, yy]
    contour(-6.0:0.1:6.0, -6.0:0.1:6.0, (x,y)->himmelblau([x, y]), fill=true, color=:deep, legend=false)
    contour!(-6.0:0.1:6.0, -6.0:0.1:6.0, (x,y)->himmelblau([x, y]), fill=false, legend=false)
    res = optimize(himmelblau, _x, BFGS(), Optim.Options(store_trace=true, extended_trace=true))
    xtracemat1 = hcat(Optim.x_trace(res)...)
    plot!(xtracemat[1, :], xtracemat[2, :], mc = :white)
    scatter!(xtracemat[1, :], xtracemat[2, :], mc = :white)
   #
    scatter!([0.], [0.], mc=:red, msc=:red)
end

Interact.Slider{Float64}(83: "input-31" = -2.0 Float64 , "x₁", -2.0, -5.0:0.5:5.0, "horizontal", true, ".3f", true)

Interact.Slider{Float64}(85: "input-32" = 0.0 Float64 , "x₂", 0.0, -5.0:0.5:5.0, "horizontal", true, ".3f", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#63146e; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.4598,343.403 56.3977,345.48 58.7142,347.687 59.4722,348.342 62.9687,351.048 63.1921,351.203 67.2232,353.738 67.8047,354.065 71.4777,355.939 73.6672,356.927 
 75.7322,357.778 79.9867,359.315 81.4909,359.788 84.2412,360.582 88.4957,361.634 92.7502,362.52 93.5031,362.65 97.0047,363.208 101.259,363.754 105.514,364.176 
 109.768,364.482 114.023,364.681 118.277,364.781 122.532,364.791 126.786,364.717 131.041,364.566 135.295,364.346 139.55,364.061 143.804,363.719 148.059,363.325 
 152.313,362.884 154.374,362.65 156.568,362.385 160.822,361.832 165.077,361.245 169.331,360.628 173.586,359.987 174.859,359.788 177.84,359.294 182.095,358.571 
 186.349,357.834 190.603,357.089 191.516,356.927 194.858,356.297 199.112,355.492 203.367,354.688 206.676,354.065 207.621,353.876 211.876,353.029 216.13,352.193 
 220.385,351.371 221.269,351.203 224.639,350.521 228.894,349.68 233.148,348.861 235.923,348.342 237.403,348.047 241.657,347.225 245.912,346.434 250.166,345.674 
 251.3,345.48 254.421,344.91 258.675,344.17 262.93,343.469 267.184,342.807 268.467,342.618 271.439,342.154 275.693,341.532 279.948,340.954 284.202,340.421 
 288.457,339.934 290.155,339.757 292.711,339.472 296.966,339.046 301.22,338.669 305.475,338.339 309.729,338.056 313.984,337.819 318.238,337.628 322.493,337.482 
 326.747,337.379 331.002,337.319 335.256,337.299 339.511,337.319 343.765,337.378 348.02,337.472 352.274,337.601 356.529,337.763 360.783,337.955 365.038,338.176 
 369.292,338.424 373.547,338.695 377.801,338.988 382.056,339.299 386.31,339.627 387.92,339.757 390.565,339.955 394.819,340.285 399.074,340.622 403.328,340.963 
 407.583,341.304 411.837,341.644 416.092,341.978 420.346,342.304 424.601,342.616 424.626,342.618 428.855,342.895 433.11,343.155 437.364,343.393 441.619,343.604 
 445.873,343.785 450.128,343.933 454.382,344.042 458.636,344.108 462.891,344.128 467.145,344.096 471.4,344.008 475.654,343.859 479.909,343.644 484.163,343.358 
 488.418,342.995 492.029,342.618 492.672,342.547 496.927,341.981 501.181,341.316 505.436,340.546 509.243,339.757 509.69,339.658 513.945,338.59 518.199,337.39 
 519.767,336.895 522.454,335.989 526.708,334.395 527.582,334.033 530.963,332.533 533.752,331.172 535.217,330.404 538.863,328.31 539.472,327.933 543.153,325.448 
 543.726,325.031 546.809,322.587 547.981,321.581 549.974,319.725 552.235,317.438 552.761,316.863 555.163,314.002 556.49,312.272 557.295,311.14 559.149,308.278 
 560.744,305.563 560.824,305.417 562.234,302.555 563.499,299.693 564.627,296.832 564.999,295.767 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#821f6b; stroke-width:1; stroke-opacity:1; fill:none" points="
 564.999,319.776 562.603,322.587 560.744,324.599 559.899,325.448 556.825,328.31 556.49,328.599 553.277,331.172 552.235,331.948 549.208,334.033 547.981,334.822 
 544.481,336.895 543.726,337.313 539.472,339.479 538.876,339.757 535.217,341.356 532.053,342.618 530.963,343.027 526.708,344.476 523.454,345.48 522.454,345.77 
 518.199,346.882 513.945,347.877 511.717,348.342 509.69,348.739 505.436,349.476 501.181,350.121 496.927,350.679 492.672,351.155 492.159,351.203 488.418,351.535 
 484.163,351.844 479.909,352.09 475.654,352.278 471.4,352.41 467.145,352.493 462.891,352.529 458.636,352.523 454.382,352.478 450.128,352.398 445.873,352.287 
 441.619,352.148 437.364,351.985 433.11,351.8 428.855,351.598 424.601,351.38 421.32,351.203 420.346,351.147 416.092,350.895 411.837,350.635 407.583,350.372 
 403.328,350.107 399.074,349.845 394.819,349.586 390.565,349.335 386.31,349.092 382.056,348.861 377.801,348.644 373.547,348.442 371.211,348.342 369.292,348.253 
 365.038,348.079 360.783,347.927 356.529,347.799 352.274,347.699 348.02,347.626 34

The lesson is clear. Non-linear functions can be a tricky thing to minimize, and the Himmelblau function is not even that hard a case! This is a very important thing to keep in mind when trying out the different methods in Optim, and wondering about inconconsistencies between runs, starting points, methods, and more.

# Syntax
To optimize (minimize) a function, you have to provide, *at the very least*, a function to calculate the objective and a starting point.
```julia
optimize(objective args..., initial_x, [method], [options])
```
some examples...

In [88]:
# Supply objective and initial point
# equivalent to:
# optimize(himmelblau, [2.0, -1.0], NelderMead(), Optim.Options())
optimize(himmelblau, [2.0,-1.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844298648286093,-1.8481263812232966]
 * Minimum: 1.236197e-10
 * Iterations: 34
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 71

In [91]:
# Supply objective, initial point and method
# equivalent to:
# optimize(himmelblau, [2.0, -1.0], BFGS(), Optim.Options()
# Notice, gradient is approximated by finite differences.
optimize(himmelblau, [2.0,-1.0], BFGS())

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.584428340262866,-1.8481265269167992]
 * Minimum: 2.504723e-19
 * Iterations: 9
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 4.25e-09 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 7.49e-13 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 35
 * Gradient Calls: 35

In [92]:
# Supply objective, gradient and initial point
# equivalent to:
# optimize(himmelblau, himmelblau_gradient!, [2.0, -1.0], BFGS(), Optim.Options())
optimize(himmelblau, himmelblau_gradient!, [2.0,-1.0])

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844283403304593,-1.8481265269644178]
 * Minimum: 6.138363e-26
 * Iterations: 8
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 2.25e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 1.25e+11 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 3.51e-12 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 34
 * Gradient Calls: 34

In [96]:
# Supply objective, gradient, initial point and method
# equivalent to:
# optimize(himmelblau, [2.0, -1.0], BFGS(), Optim.Options())
# Notice, hessian is approximated by finite differences.
optimize(himmelblau, himmelblau_gradient!, [2.0,-1.0],  Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844283403304913,-1.8481265269644038]
 * Minimum: 1.262177e-29
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 6.81e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 5.68e-14 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 31
 * Gradient Calls: 31
 * Hessian Calls: 6

In [97]:
# Supply objective, gradient, initial point
# equivalent to:
# optimize(himmelblau, himmelblau_gradient!, himmelblau_hessian!,  [2.0,-1.0],  Newton()), Optim.Options())
optimize(himmelblau, himmelblau_gradient!, himmelblau_hessian!,  [2.0,-1.0],  Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [2.0,-1.0]
 * Minimizer: [3.5844283403304913,-1.848126526964404]
 * Minimum: 1.577722e-29
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 6.83e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 5.33e-14 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 31
 * Gradient Calls: 31
 * Hessian Calls: 6